# 인공 신경망

In [ ]:
from tensorflow import keras

(train_input, train_target), (test_input, test_target) = keras.datasets.fashion_mnist.load_data()

print(train_input.shape, train_target.shape)

print(test_input.shape, test_target.shape)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

fig, axs = plt.subplots(1,10,figsize=(10,10))
for i in range(10):
    axs[i].imshow(train_input[i], cmap='gray_r')
    axs[i].axis('off')
plt.show()

print([train_target[i] for i in range(10)])

print(np.unique(train_target, return_counts=True))

In [ ]:
train_scaled = train_input / 255.0
train_scaled = train_scaled.reshape(-1,28*28)

print(train_scaled.shape)

from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import cross_validate

sc = SGDClassifier(loss='log_loss',max_iter=5, random_state=42)

scores = cross_validate(sc,train_scaled,train_target,n_jobs=-1)
print(np.mean(scores['test_score']))

In [ ]:
from sklearn.model_selection import train_test_split


train_scaled, val_scaled, train_target, val_target = train_test_split(train_scaled,train_target,test_size=0.2,random_state=42)

print(train_scaled.shape, train_target.shape)
print(val_scaled.shape, val_target.shape)

dense = keras.layers.Dense(10,activation='softmax',input_shape=(784,))

model = keras.Sequential(dense)

model.compile(loss='sparse_categorical_crossentropy',metrics='accuracy')

print(train_target[:10])

model.fit(train_scaled,train_target,epochs=5)

model.evaluate(val_scaled,val_target)

In [ ]:
# 2개의 층

dense1 = keras.layers.Dense(100,activation='sigmoid', input_shape =(784,))
dense2 = keras.layers.Dense(10,activation='softmax')

model = keras.Sequential([dense1,dense2])

model.summary()


# 위와 동일한 방법 : 아래의 방식이 주로 사용된다고 한다.
# model = keras.Sequential()
# model.add( keras.layers.Dense(100,activation='sigmoid', input_shape =(784,)))
# model.add( keras.layers.Dense(10,activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy',metrics='accuracy')

model.fit(train_scaled,train_target,epochs=5)

In [ ]:
# 렐루 함수와 Flatten 층 : 결과가 우수한데, 자세한 내용은 별도 학습이 필요하겠다.

model = keras.Sequential()
model.add(keras.layers.Flatten(input_shape=(28,28)))
model.add( keras.layers.Dense(100,activation='relu'))
model.add( keras.layers.Dense(10,activation='softmax'))
model.summary()

model.compile(loss='sparse_categorical_crossentropy',metrics='accuracy')

(train_input, train_target), (test_input, test_target) = keras.datasets.fashion_mnist.load_data()
train_scaled = train_input / 255.0
train_scaled, val_scaled, train_target, val_target = train_test_split(train_scaled,train_target,test_size=0.2,random_state=42)

model.fit(train_scaled,train_target,epochs=5)
model.evaluate(val_scaled,val_target)

In [ ]:
# 옵티마이저 

# 기본 경사 하강법 옵티마이저

# model.compile(optimizer='sgd',loss='sparse_categorical_crossentropy',metrics='accuracy')

# 위와 동일함.
sgd = keras.optimizers.SGD()
sgd = keras.optimizers.SGD(learning_rate=0.1)
sgd = keras.optimizers.SGD(momentum=0.9, nesterov=True)

model = keras.Sequential()
model.add(keras.layers.Flatten(input_shape=(28,28)))
model.add( keras.layers.Dense(100,activation='relu'))
model.add( keras.layers.Dense(10,activation='softmax'))

model.compile(optimizer=sgd, loss='sparse_categorical_crossentropy',metrics='accuracy')

model.fit(train_scaled,train_target,epochs=5)
model.evaluate(val_scaled,val_target)

In [ ]:
# 적응적 학습률 옵티마이저 : 아담

model = keras.Sequential()
model.add(keras.layers.Flatten(input_shape=(28,28)))
model.add( keras.layers.Dense(100,activation='relu'))
model.add( keras.layers.Dense(10,activation='softmax'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy',metrics='accuracy')

model.fit(train_scaled,train_target,epochs=5)
model.evaluate(val_scaled,val_target)

In [ ]:
# 손실 곡선

import tensorflow as tf

tf.keras.utils.set_random_seed(42)
tf.config.experimental.enable_op_determinism()

from tensorflow import keras
from sklearn.model_selection import train_test_split

(train_input, train_target), (test_input, test_target) = \
    keras.datasets.fashion_mnist.load_data()


train_scaled = train_input / 255.0

train_scaled, val_scaled, train_target, val_target = train_test_split(
    train_scaled, train_target, test_size=0.2, random_state=42)

def model_fn(a_layer=None):
    model = keras.Sequential()
    model.add(keras.layers.Flatten(input_shape=(28, 28)))
    model.add(keras.layers.Dense(100, activation='relu'))
    if a_layer:
        model.add(a_layer)
    model.add(keras.layers.Dense(10, activation='softmax'))
    return model

model = model_fn()
model.summary()


In [ ]:
model.compile(loss='sparse_categorical_crossentropy', metrics='accuracy')

history = model.fit(train_scaled, train_target, epochs=5, verbose=0)

print(history.history.keys())

import matplotlib.pyplot as plt

plt.plot(history.history['loss'])
plt.xlabel('epoch')
plt.ylabel('loss')
plt.show()

plt.plot(history.history['accuracy'])
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.show()

In [ ]:
model = model_fn()
model.compile(loss='sparse_categorical_crossentropy', metrics='accuracy')
history = model.fit(train_scaled, train_target, epochs=20, verbose=0)

plt.plot(history.history['loss'])
plt.xlabel('epoch')
plt.ylabel('loss')
plt.show()

In [ ]:
# 검증 손실

model = model_fn()
model.compile(loss='sparse_categorical_crossentropy', metrics='accuracy')

history = model.fit(train_scaled, train_target, epochs=20, verbose=0,
                    validation_data=(val_scaled, val_target))
print(history.history.keys())

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend(['train', 'val'])
plt.show()


In [ ]:
model = model_fn()
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy',
              metrics='accuracy')

history = model.fit(train_scaled, train_target, epochs=20, verbose=0,
                    validation_data=(val_scaled, val_target))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend(['train', 'val'])
plt.show()

In [ ]:
# 드롭 아웃 : 훈련할때 고의로 중간층을 일부 드롭해서 훈련 데이터에 과대적합 하는 것을 막는 기법

model = model_fn(keras.layers.Dropout(0.3))

model.summary()

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy',
              metrics='accuracy')

history = model.fit(train_scaled, train_target, epochs=20, verbose=0,
                    validation_data=(val_scaled, val_target))

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend(['train', 'val'])
plt.show()

In [ ]:
# 모델의 저장과 복원

model = model_fn(keras.layers.Dropout(0.3))
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy',
              metrics='accuracy')

history = model.fit(train_scaled, train_target, epochs=10, verbose=0,
                    validation_data=(val_scaled, val_target))

model.save_weights('model-weights.h5')

model.save('model-whole.h5')

model = model_fn(keras.layers.Dropout(0.3))

model.load_weights('model-weights.h5') # 모델의 가중치만 가져옴

import numpy as np

val_labels = np.argmax(model.predict(val_scaled), axis=-1)
print(np.mean(val_labels == val_target))



In [ ]:
model = keras.models.load_model('model-whole.h5') # 모델의 복원(모든 정보를 가지고 있음)

model.evaluate(val_scaled, val_target)

In [ ]:
# 콜백 : 가장 좋은 결과를 저장해 주는 클래스

model = model_fn(keras.layers.Dropout(0.3))
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy',
              metrics='accuracy')

checkpoint_cb = keras.callbacks.ModelCheckpoint('best-model.h5',save_best_only=True)

model.fit(train_scaled,train_target, epochs=20, verbose=0, validation_data=(val_scaled,val_target), callbacks=[checkpoint_cb])



In [ ]:
# Callback으로 저장한 Best model 불러오기

model = keras.models.load_model('best-model.h5')
model.evaluate(val_scaled,val_target)



In [ ]:
# call back의 조기 종료 // 증가하는 시점에 종료(그런데 patience 파라미터로 횟수 지정 가능)

model = model_fn(keras.layers.Dropout(0.3))
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy',
              metrics='accuracy')

checkpoint_cb = keras.callbacks.ModelCheckpoint('best-model.h5',
                                                save_best_only=True)
early_stopping_cb = keras.callbacks.EarlyStopping(patience=2,
                                                  restore_best_weights=True)

history = model.fit(train_scaled, train_target, epochs=20, verbose=0,
                    validation_data=(val_scaled, val_target),
                    callbacks=[checkpoint_cb, early_stopping_cb])

print(early_stopping_cb.stopped_epoch)

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend(['train', 'val'])
plt.show()